In [2]:
import pandas as pd
import numpy as np

In [3]:
all_dataset = pd.read_csv("data/arrhythmia.data", header=None)

In [4]:
#chosen_column = [0] + [i for i in range(3, 15)] + [all_dataset.columns[-1]]
chosen_column = [0] + [i for i in range(3, 13)] + [14] + [all_dataset.columns[-1]]
max_trash = 10

In [5]:
def clean_data(data):
    for col in data.columns:
        trash_row = (data[col] == "?")
        good_row = (data[col] != "?")
        if sum(trash_row) < max_trash:
            data = data[good_row]
        else:
            aver =  pd.to_numeric(data[good_row][col]).mean()
            data.loc[trash_row, col] = aver
        data[col] = pd.to_numeric(data[col])
        data[col] -= data[col].min()
    return data
        

In [6]:
#use standart skleaarn func or use your own
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#also could be a good idea to do feature scaling
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()
#scaler.fit(X_train)

#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)
#see more on https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/

In [7]:
def split_train_test(df):
    margin = int(0.8 * len(df))
    df = df.sample(frac=1)
    train = df.iloc[:margin]
    test = df.iloc[margin:]
    return train, test

In [8]:
def from_data_to_xy(data):
    x = np.array(data[data.columns[:-1]], dtype=float)
    y = np.array(data[data.columns[-1]], dtype=float)
    return x, y

GaussianNB = 0.658  || 0.5
ComplementNB = 0.536 || 0.48
MultinomialNB = 0.47  || 0.43

if delete almost emply column № 13(14), results become better
GaussianNB = 0.6832  || 0.6
ComplementNB = 0.5226 || 0.49
MultinomialNB = 0.5  || 0.45

In [9]:
from sklearn import naive_bayes

def bayes_classifier(which='gaussian'):
    classifier = {
        'gaussian' : naive_bayes.GaussianNB(),
        'complement' : naive_bayes.ComplementNB(),
        'multinomial' : naive_bayes.MultinomialNB()
    }
    clf = classifier[which]
    return clf    

In [14]:
def check_accuracy(data, classifier, num_iter=1000):
    accuracy = []
    for i in range(num_iter):
        train_data, test_data = split_train_test(data )
        train_x, train_y = from_data_to_xy(train_data)
        test_x, test_y = from_data_to_xy(test_data)
        clf = classifier.fit(train_x, train_y)
        accuracy.append(sum(clf.predict(test_x) == test_y)  / test_y.shape[0])
    return accuracy

In [11]:
dataset = all_dataset[chosen_column]
clean_dataset = clean_data(dataset)

/home/ahavriushen/Desktop/uni_proj/stat_learning/venv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [170]:
which = 'multinomial'

ac = check_accuracy(clean_data, bayes_classifier(which), 1000)
print(np.mean(ac))

0.4391573033707865


knn:
5     0.5802134831460674
6     0.5839887640449437
7     0.5853932584269662
8     0.5837415730337078
9     0.5767640449438202
10     0.5775393258426966
11     0.569752808988764
12     0.5705056179775281
13     0.5678651685393259

knn with all columns:
5     0.5801235955056179
6     0.584
7     0.5836179775280897
8     0.5845842696629213
9     0.5774494382022471
10     0.574876404494382
11     0.5708988764044943
12     0.570820224719101
13     0.5677303370786517


In [16]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=13)

for i in range(5,14):
    knn_classifier = KNeighborsClassifier(n_neighbors=i)
    ac = check_accuracy(clean_data, knn_classifier, 1000)
    print(i, "   ", np.mean(ac))

KeyboardInterrupt: 

            ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed'
    with || 
    without

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
ac = check_accuracy(clean_data, clf, 1000)
print(np.mean(ac))

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='poly')
ac = check_accuracy(clean_data, clf, 1000)
print(np.mean(ac))

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
ac = check_accuracy(clean_data, clf, 1000)
print(np.mean(ac))

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='sigmoid')
ac = check_accuracy(clean_data, clf, 1000)
print(np.mean(ac))

In [148]:
import collections

_, y_ = from_data_to_xy(clean_dataset)

print(collections.Counter(y))

print(np.unique(y))

Counter({0.0: 243, 9.0: 50, 1.0: 39, 5.0: 25, 15.0: 21, 2.0: 15, 3.0: 14, 4.0: 13, 8.0: 9, 14.0: 5, 13.0: 4, 7.0: 2, 6.0: 2})
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 13. 14. 15.]
